In [4]:
import torch
import torch.nn as nn
import os
import torch.utils.data as data
#GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))
#損失関数
criterion = nn.CrossEntropyLoss()


Using cuda device


# 今日は試しに学習させてみる


In [5]:
model = torch.hub.load('torchvggish-master', 'vggish', source='local').to(device)
#harritaylor/
#model.eval()
'''
# Download an example audio file
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)

model.forward(filename)```'''

vggish/embedding:0


'\n# Download an example audio file\nimport urllib\nurl, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")\ntry: urllib.URLopener().retrieve(url, filename)\nexcept: urllib.request.urlretrieve(url, filename)\n\nmodel.forward(filename)```'

In [6]:
class MyClassfilter(nn.Module):
    def __init__(self):
        super(MyClassfilter, self).__init__()
        self.classfilter=nn.Sequential(
            nn.Linear(128,64),
            nn.Sigmoid(),
            nn.Linear(64,1),
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.classfilter(x)
mymodel = MyClassfilter().to(device)
mymodel.eval()

MyClassfilter(
  (classfilter): Sequential(
    (0): Linear(in_features=128, out_features=64, bias=True)
    (1): Sigmoid()
    (2): Linear(in_features=64, out_features=1, bias=True)
    (3): Sigmoid()
  )
)

In [7]:
# 学習済みの重みを使用
use_pretrained = True

# モデルをロード
print(model)

VGGish(
  (features): Sequential(
    (0): Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace=True)
    (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (14): ReLU(inplace=True)
    (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False

In [8]:
#print('変更前 : ', model.embeddings[4])
#model.embeddings[4] = nn.Linear(in_features=4096, out_features=5)
#print('変更あと : ', model.embeddings[4])

In [9]:
#model.classfilter=nn.Sequential(
#    nn.Linear(128, 5)   
#)
#print(model)

In [10]:
#試しに実行
#model.eval()# 評価しない
import urllib
url, filename = ("http://soundbible.com/grab.php?id=1698&type=wav", "bus_chatter.wav")
try: urllib.URLopener().retrieve(url, filename)
except: urllib.request.urlretrieve(url, filename)
a =model.forward(filename)
print(a.size(),a.dim())
print(mymodel.forward(a))

torch.Size([19, 128]) 2
tensor([[0.5281],
        [0.5976],
        [0.4350],
        [0.5020],
        [0.6046],
        [0.4731],
        [0.4725],
        [0.5313],
        [0.5298],
        [0.5520],
        [0.4537],
        [0.4926],
        [0.4693],
        [0.4748],
        [0.4650],
        [0.5713],
        [0.5128],
        [0.5117],
        [0.4475]], device='cuda:0', grad_fn=<SigmoidBackward>)


In [11]:
#データセットの定義？
classes=["Music","other"]
dataset_dir="C:/Users/hase/git/youtubedl/result"


def make_path_list():
    train_file_list=[]
    valid_file_list=[]
    for i in range(len(classes)):
        dir_name=os.path.join(dataset_dir,classes[i]).replace("\\","/")
        file_list=os.listdir(dir_name)
        
        #8割を学習用、残りを検証用にする
        num_data = len(file_list)
        num_split = int(num_data*0.8)
        
        train_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[:num_split] ]
        valid_file_list += [[os.path.join(dir_name, file).replace('\\', '/'), i] for file in file_list[num_split:]]
    return {"train":train_file_list,"valid":valid_file_list}
# 画像データへのファイルパスとラベルを格納したリストを取得する
data_dict = make_path_list()
## リストが変かもなのでみてみる


print('学習データ数 : ', len(data_dict["train"]))
##### 先頭3件だけ表示
print(data_dict["train"][:3])

print('検証データ数 : ', len(data_dict["valid"]))
##### 先頭3件だけ表示
print(data_dict["valid"][:3])

学習データ数 :  31
[['C:/Users/hase/git/youtubedl/result/Music/--aE2O5G5WE.wav', 0], ['C:/Users/hase/git/youtubedl/result/Music/-0DLPzsiXXE.wav', 0], ['C:/Users/hase/git/youtubedl/result/Music/-0SdAVK79lg.wav', 0]]
検証データ数 :  8
[['C:/Users/hase/git/youtubedl/result/Music/-FlvaZQOr2I.wav', 0], ['C:/Users/hase/git/youtubedl/result/Music/-G_hnfp4a0M.wav', 0], ['C:/Users/hase/git/youtubedl/result/Music/-HlS6jz-f0I.wav', 0]]


In [12]:
#0830
#DataSet型を作る必要がある気がする
class MyDataset(data.Dataset):
    '''
    data_dictは[パス,番号]
    '''
    def __init__(self, data_dict,  phase='train'):
        self.data_dict = data_dict
        self.phase = phase
        
    def __len__(self):
        return len(self.data_dict)
    
    def __getitem__(self,index):
        
        wav_path,label = self.data_dict[index]
        
        return wav_path, label

#DataSetを実際に作ってみる 

train_dataset = MyDataset(
    data_dict=data_dict["train"],
    phase="train"
)

valid_dataset = MyDataset(
    data_dict=data_dict["valid"],
    phase="valid"
)

#test
print(train_dataset.__getitem__(0))


('C:/Users/hase/git/youtubedl/result/Music/--aE2O5G5WE.wav', 0)


In [13]:
#0830
#もう1手ひつようらしい

#dataloaderを用いてミニバッチを作成
batch_size=10

train_dataloader=data.DataLoader(
    train_dataset, batch_size = batch_size, shuffle=True
)
valid_dataloader=data.DataLoader(
    valid_dataset, batch_size = batch_size//2, shuffle=True
)

dataloader_dict={
    'train': train_dataloader, 
    'valid': valid_dataloader
}
print(dataloader_dict)

{'train': <torch.utils.data.dataloader.DataLoader object at 0x0000017D86D20A60>, 'valid': <torch.utils.data.dataloader.DataLoader object at 0x0000017D86D20790>}


In [14]:
# パラメータ名の確認
for name, param in model.named_parameters():
    print('name : ', name)
for prm in mymodel.named_parameters():
    print(prm)
print(mymodel.parameters())

name :  features.0.weight
name :  features.0.bias
name :  features.3.weight
name :  features.3.bias
name :  features.6.weight
name :  features.6.bias
name :  features.8.weight
name :  features.8.bias
name :  features.11.weight
name :  features.11.bias
name :  features.13.weight
name :  features.13.bias
name :  embeddings.0.weight
name :  embeddings.0.bias
name :  embeddings.2.weight
name :  embeddings.2.bias
name :  embeddings.4.weight
name :  embeddings.4.bias
name :  pproc.pca_eigen_vectors
name :  pproc.pca_means
('classfilter.0.weight', Parameter containing:
tensor([[-0.0644, -0.0225,  0.0506,  ..., -0.0287,  0.0407,  0.0386],
        [-0.0588,  0.0293, -0.0709,  ..., -0.0372,  0.0206, -0.0406],
        [ 0.0497,  0.0372,  0.0363,  ..., -0.0373, -0.0082,  0.0029],
        ...,
        [ 0.0624,  0.0098, -0.0479,  ...,  0.0564,  0.0620,  0.0603],
        [ 0.0614, -0.0287, -0.0329,  ...,  0.0278, -0.0668, -0.0570],
        [ 0.0594,  0.0862, -0.0856,  ...,  0.0474, -0.0353,  0.0609]

In [15]:
#学習させるパラメータを格納
params_to_update=[]
#学習させるパラメータ名
update_param_names=['classfilter.0.weight', 'classfilter.0.bias', 'classfilter.2.weight',
                   'classfilter.2.bias']

#対象以外は購買計算をせず、変化しないようにもする
for name,param in mymodel.named_parameters():
    if name in update_param_names:
        param.requires_grad = True
        params_to_update.append(param)
        print("name : ",name)
    else:
        param.requires_grad = False


name :  classfilter.0.weight
name :  classfilter.0.bias
name :  classfilter.2.weight
name :  classfilter.2.bias


In [16]:
import torch.optim as optim
#optimizer = optim.SGD(params_to_update, lr=0.01)
optimizer = optim.SGD(params_to_update, lr=0.01)

In [17]:
#エポック数
num_epochs=20

for epoch in range(num_epochs):
    print('Epoch {}/{}'.format(epoch+1,num_epochs))
    
    for phase in ['train','valid']:
        if phase == 'train':
            model.train()#学習モード
        else:
            model.eval()#検証？
    
        #epoch全体の損失の輪と正解数
        epoch_loss=0.0
        epoch_corrects=0
        #print(phase)
        for inputs, labels in dataloader_dict[phase]:
            #入力の確認
            #print(len(inputs),len(labels))
            #optimizer?を初期化?する?
            optimizer.zero_grad()
            
            with torch.set_grad_enabled(phase=='train'):
                #ここでエラー、ファイルパスの型を調べてみる
                #strかnp.ndarrayじゃないといけない
                
                #print(type(inputs))#<class 'tuple'>
                
                #0831入力がSTRになるようにしてみる
                #"outputs=model(inputs)
                #RuntimeError: CUDA out of memory. Tried to allocate 2.00 MiB (GPU 0; 3.00 GiB total capacity; 1.76 GiB already allocated; 1.55 MiB free; 1.87 GiB reserved in total by PyTorch)

                #outputs=[model(input) for input in inputs]
                #print(outputs)
                #tensor型にする
                #0903 毎回やってみる
             #   outputs=torch.tensor([model(input).tolist()[0] for input in inputs])
                outputs=mymodel(torch.tensor([model(input).tolist()[0] for input in inputs]).to(device))
               # print("outputs:", outputs.dim())
               # print(outputs.size())
                #損失を計算
                #ここでエラーAttributeError: 'int' object has no attribute 'size'
                #ちゃんとしたDataset肩を作って直した
               # print(outputs)#二次元tensor配列
               # print(labels)#0
                labels=labels.to(device)
                loss=criterion(outputs, labels)
                #print(outputs, labels)
                #ラベルを予測
               # _,preds = torch.max(outputs,1)
                
                
                #訓練時は逆伝搬の計算
                if phase == "train":
                    #逆伝搬
                    loss.backward()
                    
                    #パラメータ更新
                    optimizer.step()
                
                #イテレーション結果の計算
                #lossの合計を更新
                #pytorchの使用上バッチ内の平均lossが計算されているのでデータ数をかけて合計にする
                #損失和を「全データの損失/データ数」で求めるために必要らしい?
                #print(len(inputs))
                epoch_loss += loss.item()*len(inputs)
                
                #正解数の合計を更新
                epoch_corrects += torch.sum(preds == labels.data)
        #epochのlossと正解数の表示
        epoch_loss=epoch_loss/len(dataloader_dict[phase])
        epoch_acc=epoch_corrects.double()/len(dataloader_dict[phase].dataset)
        print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
    

Epoch 1/20
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0000 Acc: 1.0000
Epoch 2/20
train Loss: 0.0000 Acc: 1.0000
valid Loss: 0.0000 Acc: 1.0000
Epoch 3/20
train Loss: 0.0000 Acc: 1.0000


KeyboardInterrupt: 

In [ ]:
m = nn.Softmax(dim=2)
input = torch.randn(4, 4)
output = m(input)
print(input)
print(output)

In [18]:
m = nn.Sigmoid()
input = torch.randn(1)
output = m(input)
print(input,output)

tensor([-0.0141]) tensor([0.4965])
